# Calibrate Spectrum
This notebook shows a small example on how to modify the energy axis using the information on the core loss edge energies. Sometimes no low loss is available to know the exact offset. It could also happen that the dispersion is not entirely correct and this method could try to resolve this when the exact energies of the core loss edges are known

In [1]:
%matplotlib qt
#important for interactive plotting tools

In [2]:
import pyEELSMODEL.api as em
import copy

## Simulation of multispectrum
The multispecta will be simuluated and a wrong dispersion will be added to the multispectrum

In [3]:
from pyEELSMODEL.misc.data_simulator import simulate_data

In [4]:
hl, ll = simulate_data()

16384it [00:11, 1425.69it/s]


hll
hll
hll
hll
hll
hll


16384it [00:26, 612.74it/s]
16384it [00:00, 45905.35it/s]


Multispectrum is simulated


In [5]:
cte = 1.01 #factor to change dispersion
expected_dispersion = copy.deepcopy(hl.dispersion)
hl.dispersion = hl.dispersion * cte
ll.dispersion = ll.dispersion * cte

## Align Multispectrum
The best practice is to first align the shifts before recalibrating the energy axis.  

In [ ]:
fast_align = em.FastAlignZeroLoss(ll, other_spectra=[hl], cropping=True)
fast_align.perform_alignment()

In [ ]:
hl_al = fast_align.aligned_others[0] #the core loss multispectrum
ll_al = fast_align.aligned #the low loss 

In [ ]:
em.MultiSpectrumVisualizer([ll_al, hl_al], logscale=True)

## Calibrate spectrum
The average spectrum can be used as input while the multispectrum can be corrected simultaneously. Note that the edge are also indicated by the number and not only the letter. To be able to calibrate the energy axis, it is necessary to have at least two points (two edges).

In [ ]:
elements = ['C', 'N', 'O', 'Fe', 'Fe']
edges = ['K1', 'K1', 'K1', 'L3', 'L2']
cal = em.CalibrateSpectrum(hl_al.mean(), elements, edges, other_spectra=[hl_al, ll_al])

Shows the expected energies and if it matches with the edge onsets

In [ ]:
cal.show_real_energies()

User input where the right click indicates the energy seen on the spectrum itself. The order should match with how you input the edges itself. After clicking, a black line should appear and the selected energy of the edge is indicated. 

In [ ]:
cal.determine_edge_positions()

In [ ]:
cal.calibrate_energy_axis()

In [ ]:
cal.show_calibration_fit()

In [ ]:
cal_hlmean = cal.spectrum

In [ ]:
print('Initial dispersion was: {} eV'.format(hl_al.dispersion))
print('Updated dispersion is: {} eV'.format(cal_hlmean.dispersion))
print('Expected dispersion is: {} eV'.format(expected_dispersion)

In [ ]:
cal_hl = cal.other_spectra[0] #new updated multispectrum with right dispersion
cal_ll = cal.other_spectra[1] 

Small check can be done to see if the new energy axis is correct.

In [ ]:
cal.show_real_energies()

In order to reduce the user-input one could save the new obtained multispectrum or do the following thing

In [ ]:
cal_auto = em.CalibrateSpectrum(hl_al.mean(), elements, edges, other_spectra=[hl_al, ll_al])

#the coords obtained from the previous method 
cal_auto.coords = [284.174602224911,
 403.5955056419095,
 534.9006627834328,
 712.150902005906,
 725.4287855870457]

In [ ]:
cal_auto.calibrate_energy_axis()

In [ ]:
cal_auto.show_calibration_fit()

In [ ]:
cal_auto.show_real_energies()

In [ ]:
print('Initial dispersion was: {} eV'.format(hl_al.dispersion))
print('Updated dispersion is: {} eV'.format(cal_hlmean.dispersion))
print('Expected dispersion is: {} eV'.format(expected_dispersion)